<a href="https://colab.research.google.com/github/jatin12Sethi/Movie-recommendaton_System/blob/main/movie_reccomadation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING THE LIBRARIES


In [5]:
import numpy as np
import pandas as pd
import ast

Reading the data

In [33]:
movie=pd.read_csv("tmdb_5000_movies.csv")
credit=pd.read_csv("tmdb_5000_credits.csv")

ParserError: ignored

Shape /Dimenions of our dataset

In [9]:
print("MOVIE DATASET: ROWS x COLOUMNS")
print(movie.shape)

MOVIE DATASET: ROWS x COLOUMNS
(4803, 20)


In [10]:
print("CREDITS DATASET: ROWS x COLOUMNS")
print(credits.shape)

CREDITS DATASET: ROWS x COLOUMNS
(938, 4)


In [11]:
movie.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


Recommadation model will be based on the following parameters

In [12]:
#genres
#keywords	
#id
#release_date   
movie=movie[['id','title','overview','keywords','genres']]

In [13]:
movie.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4803 non-null   int64 
 1   title     4803 non-null   object
 2   overview  4800 non-null   object
 3   keywords  4803 non-null   object
 4   genres    4803 non-null   object
dtypes: int64(1), object(4)
memory usage: 187.7+ KB


In [14]:
movie.head(1)

,id,title,overview,keywords,genres
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [16]:
movie.isnull().sum()


id          0
title       0
overview    0
keywords    0
genres      0
dtype: int64

In [15]:
movie.dropna(inplace=True)

In [17]:
movie.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

CONVERTING A DIC INTO LIST TAKING ONLY name:

In [20]:
#for genres
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [19]:
movie['genres']=movie['genres'].apply(convert)

convert list dic into list

In [21]:
#for keyword coloum
def key_convert(obj):
  Key=[]
  for i in ast.literal_eval(obj):
    Key.append(i['name'])
  return Key


In [24]:
movie['keywords']=movie['keywords'].apply(key_convert)


In [25]:
movie['genres']=movie['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movie['overview']=movie['overview'].apply(lambda x:x.split())


In [26]:
#creating a tag cloumn
movie['tags']=movie['genres']+movie['keywords']+movie['overview']

In [27]:
movie.head(1)

,id,title,overview,keywords,genres,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[Action, Adventure, Fantasy, ScienceFiction, c..."


In [28]:
new_moviedf=movie[['id','title','tags']]

In [29]:
new_moviedf.head(1)


,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."


In [30]:
new_moviedf['tags']=new_moviedf['tags'].apply(lambda x:" ".join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
new_moviedf['tags']=new_moviedf['tags'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
new_moviedf['tags'][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [45]:
import nltk

STEMMING


In [51]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

def conv_stemmed(text):
  stemmed=[]
  for i in text.split():
    stemmed.append(ps.stem(i))
  return " ".join(stemmed)  

new_moviedf['tags']=new_moviedf['tags'].apply(conv_stemmed)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [52]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [53]:
vector=cv.fit_transform(new_moviedf['tags']).toarray()

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
similarity=cosine_similarity(vector)

In [61]:
similarity[0]

array([1.        , 0.09128709, 0.09128709, ..., 0.04622502, 0.        ,
       0.        ])

In [62]:
#recommadation Function

In [79]:
def recommadation(movie):
  movie_index=new_moviedf[new_moviedf['title']== movie].index[0]
  distances=similarity[movie_index]
  movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  print("TOP 5 Recommended Movies are:")
  for i in movie_list:
  
    print(new_moviedf.iloc[i[0]].title)

recommadation('John Carter')  

TOP 5 Recommended Movies are:
Krrish
The Other Side of Heaven
Riddick
Get Carter
The Legend of Hercules


In [75]:
new_moviedf['title']

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4800, dtype: object